In [2]:
import time
import re
import sys
import requests
import numpy as np
import pandas as pd
from pathlib import Path

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
url = 'https://www.ubereats.com/mx/feed?ps=1&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMkNhdGVkcmFsJTIwTWV0cm9wb2xpdGFuYSUyMGRlJTIwbGElMjBDaXVkYWQlMjBkZSUyME0lQzMlQTl4aWNvJTIyJTJDJTIycmVmZXJlbmNlJTIyJTNBJTIyMzNjOWJiZGItNWY3OC01MGY2LTdmNmMtNDVmNzY4MzEzOTc5JTIyJTJDJTIycmVmZXJlbmNlVHlwZSUyMiUzQSUyMnViZXJfcGxhY2VzJTIyJTJDJTIybGF0aXR1ZGUlMjIlM0ExOS40MzAxNDQ2JTJDJTIybG9uZ2l0dWRlJTIyJTNBLTk5LjEzMzkxODYlN0Q%3D'

driver = webdriver.Chrome()

driver.get(url)

In [4]:
ActionChains(driver).send_keys(Keys.END).perform()
time.sleep(5)
button = driver.find_element_by_xpath('//button[normalize-space()="Mostrar más"]')
ActionChains(driver).click(button).perform()
time.sleep(3)
ActionChains(driver).send_keys(Keys.END).perform()
time.sleep(3)

page_source = driver.page_source

In [5]:
driver.quit()

Uber = BeautifulSoup(page_source, 'lxml')

In [33]:
print(Uber.prettify())

<html class="js-focus-visible" data-rh="translate,lang" dir="ltr" lang="es-MX" translate="no">
 <head>
  <meta charset="utf-8"/>
  <title>
   Pide comida en línea | App para entrega de comida a domicilio | Uber Eats
  </title>
  <link as="script" crossorigin="anonymous" href="https://d3i4yxtzktqr9n.cloudfront.net/web-eats-v2/client-main-8d30318c94050782a8ae.js" nonce="" rel="preload"/>
  <link as="script" crossorigin="anonymous" href="https://d3i4yxtzktqr9n.cloudfront.net/web-eats-v2/client-vendor-1acfa89c68ae94c292e8.js" nonce="" rel="preload"/>
  <link as="script" crossorigin="anonymous" href="https://d3i4yxtzktqr9n.cloudfront.net/web-eats-v2/client-default-cart-mobile-cart-mobile-multistep-cart-mobile-online-ordering-checkout-desktop-multistep-roo-6210a6ce-4e20e385ba60f437bb04.js" nonce="" rel="preload"/>
  <link as="script" crossorigin="anonymous" href="https://d3i4yxtzktqr9n.cloudfront.net/web-eats-v2/client-default-active-order-mobile-root-active-order-mobile-root-online-ordering

In [36]:
# store names = h3
# delivery fee = <span class="cb ex cq b3">
# rating = <div aria-hidden="true" class="gp cc cd ec eb hp cf gh ag c0 gd">


names = Uber.select('h3')
names = [name.text for name in names]
len(names)

161

In [37]:
delivery_fee = Uber.find_all('span', {'class':'cb f4 cq b3'})
delivery_fee = [fee.text for fee in delivery_fee]
len(delivery_fee)

70